# Welcome to Colab Enterprise <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">

Connect to a Runtime and begin!

<div class="markdown-google-sans">

## **Getting started**
</div>

Set up Vertexai

In [2]:
!pip install google-cloud-aiplatform

In [4]:
! gcloud auth login


You are running on a Google Compute Engine virtual machine.
It is recommended that you use service accounts for authentication.

You can run:

  $ gcloud config set account `ACCOUNT`

to switch accounts if necessary.

Your credentials may be visible to others with access to this
virtual machine. Are you sure you want to authenticate with
your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=9pekC5eQpP2gFG1aYpLQr

In [5]:
from google.colab import auth

auth.authenticate_user()

In [6]:
PROJECT_ID = 'cd-demo-01'
REGION = 'us-central1'
import vertexai
vertexai.init(project=PROJECT_ID,
              location=REGION)

In [20]:
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm
import math
import functools
import time

def encode_texts_to_embeddings(sentences):
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception:
        return [None for _ in range(len(sentences))]

def encode_text_to_embedding_batched(sentences, api_calls_per_second = 0.33, batch_size = 5):
    # Generates batches and calls embedding API

    embeddings_list = []

    # Prepare the batches using a generator
    batches = generate_batches(sentences, batch_size)

    seconds_per_job = 1 / api_calls_per_second

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(
            batches, total = math.ceil(len(sentences) / batch_size), position=0
        ):
            futures.append(
                executor.submit(functools.partial(encode_texts_to_embeddings), batch)
            )
            time.sleep(seconds_per_job)

        for future in futures:
            embeddings_list.extend(future.result())

    is_successful = [
        embedding is not None for sentence, embedding in zip(sentences, embeddings_list)
    ]
    embeddings_list_successful = np.squeeze(
        np.stack([embedding for embedding in embeddings_list if embedding is not None])
    )
    return embeddings_list_successful

### Generate text embeddings

In [ ]:
import pandas as pd
so_df = pd.DataFrame()
so_df = pd.read_csv('/content/deploy-build-stackoverflow.csv')

In [16]:
from vertexai.language_models import TextEmbeddingModel


model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@001")
import time
import numpy as np

def generate_batches(sentences, batch_size = 5):
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]

# so_questions = so_df[0:200].input_text.tolist()
# batches = generate_batches(sentences = so_questions)

so_questions = so_df.input_text.tolist()


  0%|          | 0/80 [00:00<?, ?it/s]

NameError: ignored

### OPTIONAL: Gen embeddings

In [21]:
question_embeddings = encode_text_to_embedding_batched(
                            sentences=so_questions,
                            api_calls_per_second = 20/60,
                            batch_size = 5)

question_embeddings

  0%|          | 0/80 [00:00<?, ?it/s]

array([[-0.02668177, -0.02851841,  0.01165711, ..., -0.01442359,
        -0.0043021 , -0.06496598],
       [-0.04665147, -0.04166554,  0.03035616, ..., -0.02276639,
        -0.02370534, -0.03365194],
       [-0.00835284, -0.0044221 ,  0.02535915, ..., -0.0284955 ,
        -0.03309781, -0.053542  ],
       ...,
       [-0.00694331, -0.00619193,  0.03283683, ..., -0.0260459 ,
        -0.01802497, -0.0144918 ],
       [-0.01039375, -0.0048393 ,  0.04200063, ..., -0.0192529 ,
        -0.01571974, -0.04594716],
       [-0.01133255, -0.02716326,  0.01895321, ..., -0.02744324,
        -0.02382765, -0.05456362]])

In [22]:
import pickle
with open('/content/questions_embadddings_data.pkl', 'wb') as f:
  pickle.dump(question_embeddings, f)

### Gen search data

In [25]:
# print("Shape: " + str(so_df.shape))
# print(so_df)

so_df['embeddings'] = question_embeddings.tolist()


so_df

,input_text,output_text,embeddings
0,What IAM role does serviceaccount need for exp...,<p>I needed to activate &quot;Cloud Build API&...,"[-0.026681765913963318, -0.02851841412484646, ..."
1,Share (parameterized) cloudbuild.yaml between ...,<p>I contacted Google Cloud support and they t...,"[-0.04665147140622139, -0.04166553542017937, 0..."
2,Google Cloud Build Generating cloud_run_revisi...,<p>The log entry shown corresponds to a <code>...,"[-0.008352839387953281, -0.0044220974668860435..."
3,GCP deploy cloud function failed<p>I was tryin...,<p>We were using this command to generate the ...,"[-0.023527665063738823, -0.0423089899122715, 0..."
4,Google Cloud Build show no logs<p>I have a Goo...,"<p>I do not have all the details, so trying to...","[-0.025556528940796852, -0.0016413381090387702..."
...,...,...,...
395,Google Cloud Build how to send context with di...,<p>Posting it as an answer to give it more vis...,"[-0.04455234482884407, -0.013983490876853466, ..."
396,Why Google Cloud Build creates intermediate co...,<p>Cloud Build uses Docker to execute builds. ...,"[-0.00949002057313919, -0.017768824473023415, ..."
397,How to configure Serverless VPC Access from Cl...,<p>You are mixing 2 concepts:</p>\n<ul>\n<li>G...,"[-0.006943307816982269, -0.006191926542669535,..."
398,How to pass authenticated state from the cloud...,"<p>You need to explicitly <a href=""https://clo...","[-0.010393748059868813, -0.004839301574975252,..."


###  Demo Setup


In [28]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

from vertexai.language_models import TextEmbeddingModel
embedding_model = TextEmbeddingModel.from_pretrained(
    "textembedding-gecko@001")

from vertexai.language_models import TextGenerationModel
generation_model = TextGenerationModel.from_pretrained(
    "text-bison@001")
from IPython.display import Markdown, display
t_value = 0.2

### Demo

In [31]:
query = [' Cloud build shows no logs']

In [33]:
query_embedding = embedding_model.get_embeddings(query)[0].values
cos_sim_array = cosine_similarity([query_embedding],
                                  list(so_df.embeddings.values))
index_doc = np.argmax(cos_sim_array)
# context = so_df.input_text[index_doc] + \
# "\n Answer: " + so_df.output_text[index_doc]
# context = "Answer is "+ so_df.output_text[index_doc]

prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, answer with
             [I couldn't find a good match in the \
             document database for your query]. \
             Make the answer more general to \
             all the related questions.


             """
response = generation_model.predict(prompt = prompt,
                                    temperature = t_value,
                                    max_output_tokens = 1024)
display(Markdown(response.text))

The logs from the quickstart.sh file are not shown in the Cloud Build console because the script is not executed during the build phase. To see the output of the script, you should run the container (either locally on your computer, or via Cloud Shell). If you want to execute a script during the build phase of the container, you should use the RUN command.